# Why use DSPy

DSPy is a framework that focuses on creating GenAI applications with programming rather than prompting. At its core, it lets us create **Signatures** that allow us to provide inputs to LLMs and recieve outputs as variables. Letting us go back to good old programming

In [0]:
%pip install dspy 
dbutils.library.restartPython()

In [0]:
import dspy
from typing import Literal

First, we will set what LLM we will be using

In [0]:
llama = dspy.LM('databricks/databricks-meta-llama-3-3-70b-instruct', cache=False)
dspy.configure(lm = llama)

Lets take a look at a simple input->output example of a signature. Lets create a signature that expects a singular input and puts out a singular output

In [0]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""        
    user_question: str = dspy.InputField()
    answer: str = dspy.OutputField()

While making a signature, you can guide the input and outputs with descriptions for each input and output. Additionally, you can have a docstring for the signature as a whole.

In [0]:
generate_answer_predictor = dspy.Predict(BasicQA)
pred = generate_answer(user_question= "What is the capital of France?")
print(pred.answer)         # Look we can grab the answer from the class variable!

While this may look simple, the power lies in the way we can interact with outputs as variables. Lets see this in an example with multiple-variables being inserted and outputted!

In [0]:
class AnalyzeReview(dspy.Signature):
    """Analyze a product review to determine sentiment, extract key features, and provide a summary."""

    review_text : str = dspy.InputField()
    product_category : str = dspy.InputField()

    sentiment : Literal["positive", "negative", "neutral"] = dspy.OutputField() # We can enforce types
    key_features = dspy.OutputField()
    summary = dspy.OutputField(desc="A concise summary of the review content.")    #If we need to guide output field, we can add a description

One thing you may notice is we can encforce types. For instance
`sentiment : Literal["positive", "negative", "neutral"]` means that the output will always be one of these three options

In [0]:
analyze_review_predictor = dspy.Predict(AnalyzeReview)

ex_review_text = "I love the battery life of this camera, but the lens quality could be better."
ex_product_category = "Electronics"

analyze_results = analyze_review_predictor(
    review_text= ex_review_text,
    product_category= ex_product_category
)

print(analyze_results)

# print("Sentiment:", analyze_results.sentiment)          # Access the sentiment output
# print("Key Features:", analyze_results.key_features)    # Access the key_features output
# print("Summary:", analyze_results.summary)              # Access the summary output

In [0]:
print("Sentiment:", analyze_results.sentiment)          # Access the sentiment output
print("Key Features:", analyze_results.key_features)    # Access the key_features output
print("Summary:", analyze_results.summary)              # Access the summary output

Notice how we were able to interact with our LLM like a python function where we passed input variables and and recieved varibles out! This is where DSPy strength lies. It lets us `write programs and not prompts`

# We can use variables, so what?
Great question! Why do we even care? Well because variables lets us **Program** instead of **prompt**. This opens several doors, for instance the outputs from one signature can be used as inputs for another. Effectifly we can start defining logic!

To demostrate this. We are going to take our AnalyzeReview Signature, and use those outputs to genereate a support response.

In [0]:
class GenerateSupportResponse(dspy.Signature):
    """Generate a customer support response based on review analysis."""

    sentiment = dspy.InputField(desc="The overall sentiment from the review analysis.")
    key_features = dspy.InputField(format=list, desc="Key features extracted from the review.")
    summary = dspy.InputField(desc="Summary of the review content.")

    support_response = dspy.OutputField(desc="A customer support response addressing the review.")

generate_support_response_predictor = dspy.Predict(GenerateSupportResponse)

Lets see if this works by itself

In [0]:
generate_support_response_predictor = dspy.Predict(GenerateSupportResponse)

gen_support_response_results = generate_support_response_predictor(
    sentiment= "Positive",
    key_features= 'battery life, lens quality'
    summary= "The reviewer is satisfied with the camera's battery life but finds the lens quality to be subpar."
)

print(gen_support_response_results.support_response)


Awesome, but using hardcoded values is not programming forward. We want our inputs to be the outputs from our analzye review signature. Lets now wrap them together

In [0]:
ex_review_text = "I love this vacuum, however I wish the bag size was bigger."
ex_product_category = "Electronics"

analyze_results = analyze_review_predictor(
    review_text= ex_review_text,
    product_category= ex_product_category
)

gen_support_response_results = generate_support_response_predictor(
    sentiment=analyze_results.sentiment,
    key_features=analyze_results.key_features,
    summary=analyze_results.summary
)

print(gen_support_response_results.support_response)

# Try It Yourself

In [0]:
class <Signature_Name>(dspy.Signature):
    """<DocString>""" #Provide a docstring that informs the LLM the purpose of this signature

    <input Variable> : str = dspy.InputField() # Provide a variable name and type. Note that the LLM will use this name to understand the inputfield. You can also provide a description for the input field
    
    
    <output vairable> : str = dspy.InputField() #Same for the output field

In [0]:
dspy_predictor = dspy.predict(<Signature_Name>)

dspy_predictor = (<input Variable> = "<User Input>")

print(dspy_predictor.<output vairable>)